In [3]:
import numpy as np
import pandas as pd

import requests
import xml.etree.ElementTree as ET
from requests.utils import unquote
import re

/Users/ijongseung/house_electric/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df_seoul_energy = pd.read_csv("/Users/ijongseung/house_electric/seoul_apt.csv") # 2025_01 데이터
df_seoul_house_data1 = pd.read_csv("/Users/ijongseung/house_electric/seoul.csv")
df_seoul_house_data2 = pd.read_csv("/Users/ijongseung/house_electric/seoul2.csv")

In [4]:
def matching_COMPLEX_PK(data1, data2):
    data1 = df_seoul_house_data1
    data2 = df_seoul_house_data2
    
    # 일치시키는 데이터 매칭
    df_filtering = data2[data2["COMPLEX_PK"].isin(data1["COMPLEX_PK"])] # 상세정보가 담겨있는 데이터

    print(df_filtering)
    
    # 이후 data1[주소 있는 데이터프레임]이랑 결합
    df_filtering_merge = pd.merge(data1, df_filtering, how = "inner", on = "COMPLEX_PK")

    return df_filtering_merge

def add_column(data):
    #  열을 합쳐서 새로운 열 'concrete_address'를 추가 (결측치가 있으면 NaN 반환)
    
    data["concrete_address"] = np.where(
        data["COMPLEX_NM2"].isna() | data["DONG_NM2"].isna(), 
        np.nan, 
        data["COMPLEX_NM2"].astype(str) + " " + data["DONG_NM2"].astype(str)
    )

    data["concrete_address"] = np.where(
        data["COMPLEX_NM2"].isna() | data["DONG_NM2"].isna(), 
        np.nan, 
        data["COMPLEX_NM2"].astype(str) + " " + data["DONG_NM2"].astype(str)
    )
    return data


In [5]:
df = matching_COMPLEX_PK(df_seoul_house_data1,df_seoul_house_data2)

sample = add_column(df)

# 영등포만
sample_0dengpo = sample[sample["ADRES"].str.contains("영등포")]


        Unnamed: 0      COMPLEX_PK   DONG_NM1   DONG_NM2   DONG_NM3  \
0                0  11680120340231       210동       210동       210동   
1                1  11680120340925        101       101동       101동   
2                2  11680120340925        102       102동       102동   
3                3  11680120340925        103       103동       103동   
4                4  11680120343566  현대썬앤빌 테헤란  현대썬앤빌 테헤란  현대썬앤빌 테헤란   
...            ...             ...        ...        ...        ...   
308200      308200  11620312057450        NaN        NaN        NaN   
308201      308201  11620312057452        NaN        NaN        NaN   
308202      308202  11620312057453        NaN        NaN        NaN   
308203      308203  11620312057454        NaN        NaN        NaN   
308204      308204  11620312057455        NaN        NaN        NaN   

        GRND_FLR_CNT  
0                 12  
1                 31  
2                 31  
3                 31  
4                  7  
...      

### 네이버 지도

In [6]:
import requests

def get_road_address_naver(jibun_address, client_id, client_secret):
    """네이버 지도 API를 사용하여 지번 주소를 도로명 주소로 변환"""
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    params = {"query": jibun_address}

    try:
        response = requests.get(url, headers=headers, params=params, timeout=5)
        if response.status_code == 200:
            result = response.json()
            if result["addresses"]:
                # ✅ 도로명 주소 반환 (없으면 지번 주소 반환)
                road_address = result["addresses"][0].get("roadAddress")
                return road_address if road_address else jibun_address
            else:
                print(f"⚠️ 주소 검색 실패: {jibun_address}")
                return jibun_address
        else:
            print(f"⛔ API 요청 실패: {response.status_code}, {response.text}")
            return jibun_address

    except requests.RequestException as e:
        print(f"❌ 네트워크 오류 발생: {e}")
        return jibun_address


In [7]:
import requests
import time

def get_road_address_naver(jibun_address, client_id, client_secret, max_retries=3, timeout=20):
    """네이버 지도 API를 사용하여 지번 주소를 도로명 주소로 변환 (KeyError 방지)"""
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    params = {"query": jibun_address.strip()}

    for attempt in range(max_retries):  # ✅ 최대 3번 재시도
        try:
            response = requests.get(url, headers=headers, params=params, timeout=timeout)
            if response.status_code == 200:
                result = response.json()

                # ✅ 'addresses' 키가 없으면 기본값 반환
                if "addresses" in result and result["addresses"]:
                    return result["addresses"][0].get("roadAddress", jibun_address)
                else:
                    print(f"⚠️ 주소를 찾을 수 없음: {jibun_address}")
                    return jibun_address

            elif response.status_code == 429:
                print("⛔ 요청 제한 초과: 1초 대기 후 재시도")
                time.sleep(1)  # ✅ Rate Limit 초과 시 1초 대기
            else:
                print(f"⚠️ API 요청 실패: {response.status_code}, {response.text}")
                return jibun_address

        except requests.exceptions.Timeout:
            print(f"⏱️ 요청 시간 초과 (시도 {attempt + 1}/{max_retries}) - 재시도 중...")
            time.sleep(1)

        except requests.RequestException as e:
            print(f"❌ 네트워크 오류 발생: {e} (시도 {attempt + 1}/{max_retries})")
            time.sleep(1)

    print("⛔ 최대 재시도 횟수 초과 - 요청 실패")
    return jibun_address


In [8]:
def batch_get_road_address_naver(df, address_column, client_id, client_secret, num_threads=10):
    """DataFrame에서 지번 주소를 도로명 주소로 일괄 변환 (KeyError 방지)"""
    df = df.copy()
    addresses = df[address_column].tolist()
    results = [None] * len(addresses)

    from concurrent.futures import ThreadPoolExecutor, as_completed

    def process_address(idx, address):
        """개별 주소 처리 함수"""
        results[idx] = get_road_address_naver(address, client_id, client_secret)

    # ✅ 멀티스레딩 적용
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(process_address, idx, address) for idx, address in enumerate(addresses)]
        for future in as_completed(futures):
            future.result()

    df["도로명주소"] = results
    return df


In [58]:
CLIENT_ID = "2zqidx6s39"
CLIENT_SECRET = "hEQAw9jnXJFygpFHuOhOZmciUrqjupiKEtfFu5XC"

result = batch_get_road_address_naver(sample_0dengpo, "ADRES", CLIENT_ID, CLIENT_SECRET, num_threads=5)

In [61]:
result.drop("doro_juso", inplace=True, axis=1)

In [63]:
result.to_csv("영등포_250222.csv")

### 여기서

In [4]:
# result ->  DONG_NM2 : 102동이면 주소 열을 하나 더 파서 주소에 표시할것
result = pd.read_csv("/Users/ijongseung/house_electric/영등포_250222.csv", index_col=False)

In [5]:
def create_display_address(row):
    """도로명주소와 DONG_NM2를 결합 (조건에 맞게 처리)"""
    dong_nm = row["DONG_NM2"] if pd.notna(row["DONG_NM2"]) else ""
    road_address = row["도로명주소"]

    if dong_nm == "" or dong_nm in road_address:
        return road_address
    else:
        return road_address + " " + dong_nm

result["display_address"] = result.apply(create_display_address, axis=1)

In [9]:
result["ADRES"]

0         서울특별시 영등포구 신길동 227-12
1         서울특별시 영등포구 신길동 227-12
2         서울특별시 영등포구 대림동 851-73
3         서울특별시 영등포구 대림동 851-73
4         서울특별시 영등포구 당산동4가 27-5
                  ...          
10979      서울특별시 영등포구 신길동 95-72
10980    서울특별시 영등포구 대림동 1011-18
10981    서울특별시 영등포구 대림동 1011-18
10982    서울특별시 영등포구 양평동4가 179-1
10983    서울특별시 영등포구 양평동4가 179-1
Name: ADRES, Length: 10984, dtype: object

In [10]:
def get_lat_lon_naver(jibun_address, client_id, client_secret):
    """네이버 지오코딩 API를 사용하여 지번주소 → 위도/경도 변환"""
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    params = {"query": jibun_address}

    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        if response.status_code == 200:
            result = response.json()
            if result.get("addresses"):
                lat = float(result["addresses"][0]["y"])
                lon = float(result["addresses"][0]["x"])
                print(f"✅ 지번주소 변환 성공: {jibun_address} → 위도: {lat}, 경도: {lon}")
                return lat, lon
            else:
                print(f"⚠️ 지번주소를 찾을 수 없음: {jibun_address}")
                return None, None
        else:
            print(f"⛔ API 요청 실패: {response.status_code}, {response.text}")
            return None, None

    except requests.exceptions.RequestException as e:
        print(f"❌ 네트워크 오류 발생: {e}")
        return None, None

In [17]:
def get_lat_lon_naver(address, client_id, client_secret, max_retries=3):
    """네이버 지오코딩 API를 사용하여 위도/경도 변환"""
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    params = {"query": address.strip()}

    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, params=params, timeout=10)
            if response.status_code == 200:
                result = response.json()
                if result.get("addresses"):
                    lat = float(result["addresses"][0]["y"])
                    lon = float(result["addresses"][0]["x"])
                    return lat, lon
                else:
                    print(f"⚠️ 주소를 찾을 수 없음: {address}")
                    return None, None
            else:
                print(f"⛔ API 요청 실패: {response.status_code}, {response.text}")
                time.sleep(1)  # 1초 대기 후 재시도

        except requests.exceptions.Timeout:
            print(f"❌ 요청 시간 초과 (재시도 {attempt + 1}/{max_retries})")
            time.sleep(2)  # 타임아웃 시 2초 대기

    print(f"❌ 요청 실패: {address}")
    return None, None


In [13]:
import pandas as pd
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_lat_lon_naver(jibun_address, client_id, client_secret):
    """네이버 지오코딩 API를 사용하여 지번주소 → 위도/경도 변환"""
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    params = {"query": jibun_address}

    for _ in range(3):  # ✅ 최대 3번 재시도
        try:
            response = requests.get(url, headers=headers, params=params, timeout=5)
            if response.status_code == 200:
                result = response.json()
                if result.get("addresses"):
                    lat = float(result["addresses"][0]["y"])
                    lon = float(result["addresses"][0]["x"])
                    return lat, lon
                else:
                    return None, None
            elif response.status_code == 429:
                print("⛔ API 요청 제한 초과, 1초 대기 중...")
                time.sleep(1)  # ✅ 요청 제한 초과 시 1초 대기
            else:
                print(f"⚠️ 요청 실패: {response.status_code}, {response.text}")
                return None, None
        except requests.RequestException as e:
            print(f"❌ 네트워크 오류 발생: {e}")
            time.sleep(1)

    return None, None


def batch_get_lat_lon(df, address_column, client_id, client_secret, num_threads=10):
    """DataFrame의 지번주소를 멀티스레드로 처리하여 위/경도 추가"""
    results = [None] * len(df)

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = {
            executor.submit(get_lat_lon_naver, address, client_id, client_secret): idx
            for idx, address in enumerate(df[address_column])
        }

        for future in as_completed(futures):
            idx = futures[future]
            try:
                results[idx] = future.result()
            except Exception as e:
                print(f"⚠️ 스레드 오류 발생: {e}")
                results[idx] = (None, None)

    # ✅ 결과를 데이터프레임에 추가
    df[["lat", "lon"]] = pd.DataFrame(results, index=df.index)
    return df


In [94]:
# ✅ 네이버 API 인증 정보
CLIENT_ID = "2zqidx6s39"
CLIENT_SECRET = "hEQAw9jnXJFygpFHuOhOZmciUrqjupiKEtfFu5XC"

# ✅ 멀티스레드로 데이터 처리
df = batch_get_lat_lon(result, "ADRES", CLIENT_ID, CLIENT_SECRET, num_threads=10)

# ✅ 결과 확인
print(df)

❌ 네트워크 오류 발생: HTTPSConnectionPool(host='naveropenapi.apigw.ntruss.com', port=443): Max retries exceeded with url: /map-geocode/v2/geocode?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%98%81%EB%93%B1%ED%8F%AC%EA%B5%AC+%EB%8C%80%EB%A6%BC%EB%8F%99+1072-39 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1357998e0>: Failed to resolve 'naveropenapi.apigw.ntruss.com' ([Errno 8] nodename nor servname provided, or not known)"))
       Unnamed: 0  Unnamed: 0_x                   ADRES  COMPLEX_GB_CD  \
0          335960        139515   서울특별시 영등포구 신길동 227-12              3   
1          335961        139515   서울특별시 영등포구 신길동 227-12              3   
2          335962        139516   서울특별시 영등포구 대림동 851-73              3   
3          335963        139516   서울특별시 영등포구 대림동 851-73              3   
4          335964        139517   서울특별시 영등포구 당산동4가 27-5              3   
...           ...           ...                     ...            ...   
10979      405113

In [167]:
import pandas as pd
import json

def generate_naver_map_html(df, client_id, output_filename="naver_map.html"):
    """네이버 지도 - 검색하면 마커가 표시되는 기능"""
    
    # ✅ 지도 중심 좌표 설정
    center_lat = df["lat"].mean() if not df.empty else 37.507975
    center_lon = df["lon"].mean() if not df.empty else 126.910324

    # ✅ 데이터프레임을 JSON으로 변환
    markers = df.to_dict(orient="records")

    # ✅ HTML 파일 생성
    html_template = f"""
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <title>네이버 지도 - 검색 시 마커 표시</title>
        <script type="text/javascript" src="https://oapi.map.naver.com/openapi/v3/maps.js?ncpClientId={client_id}"></script>
        
        <style>
            body {{ font-family: Arial, sans-serif; }}
            #map {{ width: 100%; height: 700px; }}
            #searchBox {{
                position: absolute;
                top: 20px;
                left: 20px;
                width: 300px;
                padding: 10px;
                background: white;
                border: 1px solid #ccc;
                border-radius: 10px;
                z-index: 1000;
            }}
            #searchInput {{
                width: 100%;
                padding: 8px;
                font-size: 16px;
                border: 1px solid #ccc;
                border-radius: 5px;
            }}
            #suggestions {{
                list-style: none;
                padding: 0;
                margin: 5px 0 0 0;
                border: 1px solid #ccc;
                border-radius: 5px;
                overflow: hidden;
            }}
            #suggestions li {{
                padding: 8px;
                cursor: pointer;
                border-bottom: 1px solid #eee;
            }}
            #suggestions li:hover {{
                background: #e0f7fa;
            }}
        </style>
    </head>

    <body>
        <!-- ✅ 지도 표시 -->
        <div id="map"></div>

        <!-- ✅ 검색창 및 자동완성 -->
        <div id="searchBox">
            <input type="text" id="searchInput" placeholder="주소를 입력하세요...">
            <ul id="suggestions"></ul>
        </div>

        <script>
            // ✅ 지도 초기화
            var mapOptions = {{
                center: new naver.maps.LatLng({center_lat}, {center_lon}),
                zoom: 12
            }};
            var map = new naver.maps.Map('map', mapOptions);

            // ✅ 데이터 로드
            var locations = {json.dumps(markers, ensure_ascii=False)};
            var currentMarker = null;

            // ✅ 검색 및 자동완성 기능
            var searchInput = document.getElementById('searchInput');
            var suggestions = document.getElementById('suggestions');

            searchInput.addEventListener("input", function () {{
                var inputValue = this.value.toLowerCase().trim();
                suggestions.innerHTML = "";

                if (inputValue !== "") {{
                    var filtered = locations.filter(function(item) {{
                        return item.display_address && item.display_address.toLowerCase().includes(inputValue);
                    }});

                    filtered.slice(0, 5).forEach(function(item) {{
                        var li = document.createElement("li");
                        li.textContent = item.display_address;
                        li.onclick = function () {{
                            searchInput.value = item.display_address;
                            suggestions.innerHTML = "";
                            showMarker(item);
                        }};
                        suggestions.appendChild(li);
                    }});
                }}
            }});

            // ✅ 마커 표시 함수
            function showMarker(item) {{
                // 기존 마커 제거
                if (currentMarker) {{
                    currentMarker.setMap(null);
                }}

                // 새 마커 추가
                currentMarker = new naver.maps.Marker({{
                    position: new naver.maps.LatLng(item.lat, item.lon),
                    map: map,
                    title: item.display_address
                }});

                var infoWindow = new naver.maps.InfoWindow({{
                    content: `<div style="padding:8px; font-size:14px;">
                                <b>${{item.display_address}}</b><br>
                              </div>`
                }});

                naver.maps.Event.addListener(currentMarker, "click", function() {{
                    infoWindow.open(map, currentMarker);
                }});

                // 지도 이동
                map.setCenter(new naver.maps.LatLng(item.lat, item.lon));
                map.setZoom(15);
            }}
        </script>
    </body>
    </html>
    """

    # ✅ HTML 파일 저장
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(html_template)

    print(f"✅ 네이버 지도 HTML 파일이 생성되었습니다: {output_filename}")


In [168]:
generate_naver_map_html(df1, "2zqidx6s39", output_filename="naver_map.html")

✅ 네이버 지도 HTML 파일이 생성되었습니다: naver_map.html


In [170]:
df1.to_csv("영등포_위경도.csv")

## 영등포 데이터 추출

In [388]:
df2 = pd.read_csv("/Users/ijongseung/house_electric/seoul_apt.csv")

In [389]:
df2_0deungop = df2[df2["kaptAddr"].str.contains("영등포")]

In [390]:
df2_0deungop

,Unnamed: 0,bjdCode,codeAptNm,codeHallNm,codeHeatNm,codeMgrNm,codeSaleNm,doroJuso,hoCnt,kaptAcompany,...,가스사용금액,난방사용금액,전기사용량,가스사용량,난방사용량,냉수사용량,온수사용량,냉수사용금액,온수사용금액,집계날짜
36,36,1156013200,아파트,계단식,개별난방,위탁관리,분양,NaN,134.0,대성산업,...,0,0,0,0,0,0,0,0,0,202501
43,43,1156013300,아파트,계단식,개별난방,위탁관리,분양,NaN,133.0,문영건설,...,0,0,0,0,0,0,0,0,0,202501
45,45,1156013300,아파트,혼합식,개별난방,위탁관리,분양,NaN,143.0,대림연립재건축조합,...,0,0,0,0,0,0,0,0,0,202501
47,47,1156013200,아파트,복도식,개별난방,위탁관리,임대,NaN,576.0,(주)랜드코퍼레이션,...,0,0,0,0,0,0,0,0,0,202501
63,63,1156012700,아파트,계단식,개별난방,위탁관리,분양,NaN,116.0,"유성,장수연립재건축조합",...,0,0,0,0,0,0,0,0,0,202501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647,2647,1156011000,아파트,계단식,지역난방,자치관리,분양,서울특별시 영등포구 63로 7,360.0,한양주택개발(주),...,0,0,0,0,0,0,0,0,0,202501
2648,2648,1156011000,아파트,복도식,지역난방,자치관리,분양,서울특별시 영등포구 국제금융로 108-6,376.0,라이프주택개발(주),...,0,63507410,116902,0,0,4149,1079,5102680,4316000,202501
2649,2649,1156013300,아파트,혼합식,개별난방,자치관리,분양,서울특별시 영등포구 시흥대로187길 12,217.0,대림동지역주택조합,...,0,0,0,0,0,0,0,0,0,202501
2650,2650,1156013300,아파트,계단식,개별난방,위탁관리,분양,서울특별시 영등포구 도림로54길 14,185.0,대림2주택조합,...,0,0,62186,0,0,3109,0,3682620,0,202501


In [391]:
import json
import requests
import pandas as pd
import re

class get_geo_data_naver:
    def __init__(self, NAVER_CLIENT_ID, NAVER_CLIENT_SECRET, district_name):
        self.NAVER_CLIENT_ID = NAVER_CLIENT_ID
        self.NAVER_CLIENT_SECRET = NAVER_CLIENT_SECRET
        self.district_name = district_name

    def get_district_boundary(self, district_name):
        with open("/Users/ijongseung/house_electric/html/line_data.json", "r", encoding="utf-8") as f:
            districts_data = json.load(f)
        
        for district in districts_data["features"]:
            if district["properties"]["name"] == district_name:
                return district["geometry"]["coordinates"][0]
        return []

    def clean_address(self, address):
        if pd.isna(address):
            return address
        return re.sub(r"(\d+-?\d*)\\s?.*", r"\\1", address).strip()

    def get_road_address(self, jibun_address):
        url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.NAVER_CLIENT_ID,
            "X-NCP-APIGW-API-KEY": self.NAVER_CLIENT_SECRET
        }
        params = {"query": jibun_address}

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result.get("addresses"):
                road_address = result["addresses"][0].get("roadAddress")
                if road_address:
                    return road_address
        return jibun_address

    def get_lat_lon(self, address):
        url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.NAVER_CLIENT_ID,
            "X-NCP-APIGW-API-KEY": self.NAVER_CLIENT_SECRET
        }
        params = {"query": address}

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result.get("addresses"):
                lat = float(result["addresses"][0]["y"])
                lon = float(result["addresses"][0]["x"])
                return lat, lon
        return None, None

    def preprocessing(self, df):
        df_revision = df[df["kaptAddr"].str.contains(self.district_name)].copy()
        df_revision.rename(columns={"doroJuso": "address"}, inplace=True)

        df_revision["cleaned_address"] = df_revision["kaptAddr"].apply(self.clean_address)

        df_revision["address"] = df_revision.apply(
            lambda row: self.get_road_address(row["cleaned_address"]) if pd.isna(row.get("address")) else row["address"],
            axis=1
        )

        df_revision[["lat", "lon"]] = df_revision["address"].apply(
            lambda x: self.get_lat_lon(x) if pd.notna(x) else (None, None)
        ).apply(pd.Series)

        df_filtered = df_revision.dropna(subset=["lat", "lon"])

        if "전기사용량" not in df_filtered.columns:
            df_filtered["전기사용량"] = 0

        return df_filtered

    def generate_naver_map_html(self, df, client_id, output_filename="map.html"):
        district_boundary = self.get_district_boundary(self.district_name)
        if not district_boundary:
            return

        df_filtered = self.preprocessing(df)

        df_filtered = df_filtered.rename(columns={"address": "address_en", "전기사용량": "electricity_usage"})
        markers_data = df_filtered.to_dict(orient="records")

        html_template = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <meta charset="utf-8"/>
            <title>네이버 지도 - {self.district_name}</title>
            <script type="text/javascript" src="https://oapi.map.naver.com/openapi/v3/maps.js?ncpClientId={client_id}"></script>
        </head>
        <body>
            <div id="map" style="width:100%;height:600px;"></div>
            <script>
                var map = new naver.maps.Map(document.getElementById('map'), {{
                    center: new naver.maps.LatLng({df_filtered['lat'].mean()}, {df_filtered['lon'].mean()}),
                    zoom: 12
                }});

                var districtBoundary = {json.dumps(district_boundary)};
                var polygonPath = districtBoundary.map(function(coord) {{
                    return new naver.maps.LatLng(coord[1], coord[0]);
                }});

                var polygon = new naver.maps.Polygon({{
                    paths: polygonPath,
                    strokeWeight: 3,
                    strokeColor: '#FF0000',
                    strokeOpacity: 0.8,
                    fillColor: '#FF0000',
                    fillOpacity: 0.3
                }});
                polygon.setMap(map);

                var markers = {json.dumps(markers_data, ensure_ascii=False)};

                markers.forEach(function(markerData) {{
                    var marker = new naver.maps.Marker({{
                        map: map,
                        position: new naver.maps.LatLng(markerData.lat, markerData.lon)
                    }});

                    var infoWindow = new naver.maps.InfoWindow({{
                        content: '<div style="padding:5px;"><b>주소:</b> ' + markerData.address_en + '<br>' +
                                 '<b>전기사용량:</b> ' + markerData.electricity_usage + ' kWh</div>'
                    }});

                    naver.maps.Event.addListener(marker, 'click', function() {{
                        infoWindow.open(map, marker);
                    }});
                }});
            </script>
        </body>
        </html>
        """

        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(html_template)

        print(f"✅ '{self.district_name}' 구역의 네이버 지도 HTML 파일이 생성되었습니다: http://localhost:8080/{output_filename}")


In [392]:
# ✅ 네이버 API 인증 정보
CLIENT_ID = "2zqidx6s39"
CLIENT_SECRET = "hEQAw9jnXJFygpFHuOhOZmciUrqjupiKEtfFu5XC"

In [393]:
geo_data = get_geo_data_naver(NAVER_CLIENT_ID=CLIENT_ID, 
                              NAVER_CLIENT_SECRET=CLIENT_SECRET, 
                              district_name="영등포구")

In [394]:
geo_data.generate_naver_map_html(df2_0deungop, client_id=CLIENT_ID)
Dudemdvh = geo_data.preprocessing(df2_0deungop)

✅ '영등포구' 구역의 네이버 지도 HTML 파일이 생성되었습니다: http://localhost:8080/map.html


KeyboardInterrupt: 

In [420]:
address.iloc[:, 0]

36      서울특별시 영등포구 가마산로50길 27 신길동 대성 유니드 아파트
43          서울특별시 영등포구 도림천로 391 대림동문영칸타빌레아파트
45          서울특별시 영등포구 디지털로69길 13 대림동1차한신아파트
47                   서울특별시 영등포구 신풍로 28 비스타동원
63                       서울특별시 영등포구 선유서로 116
                        ...                 
2647                        서울특별시 영등포구 63로 7
2648                  서울특별시 영등포구 국제금융로 108-6
2649                  서울특별시 영등포구 시흥대로187길 12
2650                    서울특별시 영등포구 도림로54길 14
2651                   서울특별시 영등포구 가마산로79길 19
Name: address, Length: 178, dtype: object

In [423]:
lat_lot = Dudemdvh[["lat", "lon"]]
# address = Dudemdvh["address"]
# Dudemdvh.drop("address", inplace=True, axis=1)

Dudemdvh["address"] = address.iloc[:, 0]

In [424]:
Dudemdvh

,Unnamed: 0,bjdCode,codeAptNm,codeHallNm,codeHeatNm,codeMgrNm,codeSaleNm,hoCnt,kaptAcompany,kaptAddr,...,가스사용량,난방사용량,냉수사용량,온수사용량,냉수사용금액,온수사용금액,집계날짜,lat,lon,address
36,36,1156013200,아파트,계단식,개별난방,위탁관리,분양,134.0,대성산업,서울특별시 영등포구 신길동 389- 신길동대성유니드아파트,...,0,0,0,0,0,0,202501,37.501687,126.902830,서울특별시 영등포구 가마산로50길 27 신길동 대성 유니드 아파트
43,43,1156013300,아파트,계단식,개별난방,위탁관리,분양,133.0,문영건설,서울특별시 영등포구 대림동 1061- 문영칸타빌레아파트,...,0,0,0,0,0,0,202501,37.489759,126.897556,서울특별시 영등포구 도림천로 391 대림동문영칸타빌레아파트
45,45,1156013300,아파트,혼합식,개별난방,위탁관리,분양,143.0,대림연립재건축조합,서울특별시 영등포구 대림동 887-1 대림동1차한신아파트,...,0,0,0,0,0,0,202501,37.494787,126.903399,서울특별시 영등포구 디지털로69길 13 대림동1차한신아파트
47,47,1156013200,아파트,복도식,개별난방,위탁관리,임대,576.0,(주)랜드코퍼레이션,서울특별시 영등포구 신길동 4975- 신풍역 비스타동원,...,0,0,0,0,0,0,202501,37.499810,126.909558,서울특별시 영등포구 신풍로 28 비스타동원
63,63,1156012700,아파트,계단식,개별난방,위탁관리,분양,116.0,"유성,장수연립재건축조합",서울특별시 영등포구 양평동3가 19- 양평역월드메르디앙아파트,...,0,0,0,0,0,0,202501,37.526055,126.886739,서울특별시 영등포구 선유서로 116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647,2647,1156011000,아파트,계단식,지역난방,자치관리,분양,360.0,한양주택개발(주),서울특별시 영등포구 여의도동 52- 여의도은하,...,0,0,0,0,0,0,202501,37.518935,126.935337,서울특별시 영등포구 63로 7
2648,2648,1156011000,아파트,복도식,지역난방,자치관리,분양,376.0,라이프주택개발(주),서울특별시 영등포구 여의도동 54- 여의도진주,...,0,0,4149,1079,5102680,4316000,202501,37.518342,126.932948,서울특별시 영등포구 국제금융로 108-6
2649,2649,1156013300,아파트,혼합식,개별난방,자치관리,분양,217.0,대림동지역주택조합,서울특별시 영등포구 대림동 946-1 대림동현대,...,0,0,0,0,0,0,202501,37.492921,126.908391,서울특별시 영등포구 시흥대로187길 12
2650,2650,1156013300,아파트,계단식,개별난방,위탁관리,분양,185.0,대림2주택조합,서울특별시 영등포구 대림동 1124- 신대림신동아파밀리에,...,0,0,3109,0,3682620,0,202501,37.494884,126.902681,서울특별시 영등포구 도림로54길 14


In [425]:
import json
import requests
import pandas as pd
import re

class get_geo_data_naver:
    def __init__(self, NAVER_CLIENT_ID, NAVER_CLIENT_SECRET, district_name):
        self.NAVER_CLIENT_ID = NAVER_CLIENT_ID
        self.NAVER_CLIENT_SECRET = NAVER_CLIENT_SECRET
        self.district_name = district_name

    def get_lat_lon(self, address):
        url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.NAVER_CLIENT_ID,
            "X-NCP-APIGW-API-KEY": self.NAVER_CLIENT_SECRET
        }
        params = {"query": address}

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result.get("addresses"):
                lat = float(result["addresses"][0]["y"])
                lon = float(result["addresses"][0]["x"])
                return lat, lon
        return None, None

    def generate_naver_map_html(self, df, client_id, output_filename="map.html"):
        center_lat = df["lat"].mean() if not df.empty else 37.507975
        center_lon = df["lon"].mean() if not df.empty else 126.910324

        markers = df.to_dict(orient="records")

        html_template = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <meta charset="utf-8"/>
            <title>네이버 지도 - 검색 기능</title>
            <script type="text/javascript" src="https://oapi.map.naver.com/openapi/v3/maps.js?ncpClientId={client_id}"></script>
            <style>
                #map {{ width: 100%; height: 600px; }}
                #searchBox {{
                    position: absolute;
                    top: 10px;
                    left: 10px;
                    width: 300px;
                    background: white;
                    border: 1px solid #ccc;
                    border-radius: 5px;
                    z-index: 1000;
                    padding: 10px;
                }}
                #searchInput {{
                    width: 100%;
                    padding: 8px;
                    border: 1px solid #ccc;
                    border-radius: 5px;
                }}
                #suggestions {{
                    list-style: none;
                    padding: 0;
                    margin: 5px 0 0 0;
                    border: 1px solid #ccc;
                    border-radius: 5px;
                    overflow: hidden;
                }}
                #suggestions li {{
                    padding: 8px;
                    cursor: pointer;
                    border-bottom: 1px solid #eee;
                }}
                #suggestions li:hover {{
                    background: #e0f7fa;
                }}
            </style>
        </head>
        <body>
            <div id="map"></div>
            <div id="searchBox">
                <input type="text" id="searchInput" placeholder="주소를 검색하세요...">
                <ul id="suggestions"></ul>
            </div>

            <script>
                var map = new naver.maps.Map('map', {{
                    center: new naver.maps.LatLng({center_lat}, {center_lon}),
                    zoom: 12
                }});

                var markers = {json.dumps(markers, ensure_ascii=False)};
                var currentMarker = null;
                var searchInput = document.getElementById('searchInput');
                var suggestions = document.getElementById('suggestions');

                searchInput.addEventListener('input', function () {{
                    var query = this.value.toLowerCase().trim();
                    suggestions.innerHTML = '';

                    if (query) {{
                        var filtered = markers.filter(function(item) {{
                            return item.address_en && item.address_en.toLowerCase().includes(query);
                        }});

                        filtered.slice(0, 5).forEach(function(item) {{
                            var li = document.createElement('li');
                            li.textContent = item.address_en;
                            li.onclick = function () {{
                                searchInput.value = item.address_en;
                                suggestions.innerHTML = '';
                                showMarker(item);
                            }};
                            suggestions.appendChild(li);
                        }});
                    }}
                }});

                function showMarker(item) {{
                    if (currentMarker) {{
                        currentMarker.setMap(null);
                    }}

                    currentMarker = new naver.maps.Marker({{
                        position: new naver.maps.LatLng(item.lat, item.lon),
                        map: map
                    }});

                    var infoWindow = new naver.maps.InfoWindow({{
                        content: `<div style="padding:5px;"><b>주소:</b> ${item.address_en}<br>
                                  <b>위도:</b> ${item.lat}<br>
                                  <b>경도:</b> ${item.lon}<br>
                                  <b>전기사용량:</b> ${item.electricity_usage} kWh</div>`
                    }});

                    naver.maps.Event.addListener(currentMarker, 'click', function() {{
                        infoWindow.open(map, currentMarker);
                    }});

                    map.setCenter(new naver.maps.LatLng(item.lat, item.lon));
                    map.setZoom(15);
                }}
            </script>
        </body>
        </html>
        """

        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(html_template)

        print(f"✅ 네이버 지도 HTML 파일이 생성되었습니다: {output_filename}")


In [426]:
geo_data1 = get_geo_data_naver(NAVER_CLIENT_ID=CLIENT_ID, 
                              NAVER_CLIENT_SECRET=CLIENT_SECRET, 
                              district_name="영등포구")

In [248]:
dudemdvh= geo_data.generate_naver_map_html(df2_0deungop, client_id=CLIENT_ID)

✅ '영등포구' 구역의 네이버 지도 HTML 파일이 생성되었습니다: http://localhost:8080/map.html


In [431]:
 import json
import requests
import pandas as pd
import re
from difflib import SequenceMatcher

class get_geo_data_naver:
    def __init__(self, NAVER_CLIENT_ID, NAVER_CLIENT_SECRET, district_name):
        self.NAVER_CLIENT_ID = NAVER_CLIENT_ID
        self.NAVER_CLIENT_SECRET = NAVER_CLIENT_SECRET
        self.district_name = district_name

    def similar(self, a: str, b: str) -> float:
        """두 문자열의 유사도를 0~1 사이로 계산하여 리턴 (1에 가까울수록 유사)."""
        return SequenceMatcher(None, a, b).ratio()

    def merge_data(self, df_energy: pd.DataFrame, df_building: pd.DataFrame, similarity_threshold: float = 0.6) -> pd.DataFrame:
        """
        주소를 기준으로 df_energy와 df_building을 유사도 매칭하여 병합.
        similarity_threshold 이상일 때 전력소비량(electricity_usage)을 df_building에 매핑한다.
        """
        if "electricity_use" not in df_energy.columns:
            df_building["electricity_use"] = 0

        for idx_b, building_row in df_building.iterrows():
            building_address = str(building_row["address"])
            max_similarity = 0.0
            matched_usage = 0

            for idx_e, energy_row in df_energy.iterrows():
                energy_address = str(energy_row["address"])
                sim = self.similar(building_address, energy_address)
                if sim > max_similarity and sim >= similarity_threshold:
                    max_similarity = sim
                    matched_usage = energy_row.get("electricity_use", 0)

            df_building.at[idx_b, "electricity_use"] = matched_usage

        return df_building

    def clean_address(self, address: str) -> str:
        """주소에서 번지 추출 등 간단 전처리"""
        if pd.isna(address):
            return address
        return re.sub(r"(\d+-?\d*)\s?.*", r"\1", address).strip()

    def get_road_address(self, jibun_address: str) -> str:
        """
        지번주소(jibun_address)를 네이버 지오코드 API로 도로명주소로 변환.
        변환 실패 시 원본 주소를 반환.
        """
        url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.NAVER_CLIENT_ID,
            "X-NCP-APIGW-API-KEY": self.NAVER_CLIENT_SECRET
        }
        params = {"query": jibun_address}

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result.get("addresses"):
                road_address = result["addresses"][0].get("roadAddress")
                if road_address:
                    return road_address
        return jibun_address

    def get_lat_lon(self, address: str):
        """주소를 위도(lat), 경도(lon)로 변환. 실패 시 (None, None) 반환."""
        url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.NAVER_CLIENT_ID,
            "X-NCP-APIGW-API-KEY": self.NAVER_CLIENT_SECRET
        }
        params = {"query": address}

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result.get("addresses"):
                lat = float(result["addresses"][0]["y"])
                lon = float(result["addresses"][0]["x"])
                return lat, lon
        return None, None

    def preprocessing(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        1) district_name이 포함된 주소 필터
        2) 주소 클리닝 & 도로명주소 변환
        3) 위경도 변환
        4) 결측치 제거
        """
        df_revision = df[df["address"].str.contains(self.district_name, na=False)].copy()

        df_revision["cleaned_address"] = df_revision["address"].apply(self.clean_address)
        df_revision["address"] = df_revision.apply(
            lambda row: self.get_road_address(row["cleaned_address"]) if pd.isna(row["address"]) else row["address"],
            axis=1
        )

        df_revision[["lat", "lon"]] = df_revision["address"].apply(
            lambda x: self.get_lat_lon(x) if pd.notna(x) else (None, None)
        ).apply(pd.Series)

        df_filtered = df_revision.dropna(subset=["lat", "lon"])

        if "electricity_use" not in df_filtered.columns:
            df_filtered["electricity_use"] = 0

        return df_filtered

In [432]:
print(Dudemdvh["electricity_use"].describe())
print(Dudemdvh[["address", "electricity_use"]].head(50))

count       178.000000
mean      24120.292135
std       75758.577723
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      584228.000000
Name: electricity_use, dtype: float64
                                  address  electricity_use
36   서울특별시 영등포구 가마산로50길 27 신길동 대성 유니드 아파트                0
43       서울특별시 영등포구 도림천로 391 대림동문영칸타빌레아파트                0
45       서울특별시 영등포구 디지털로69길 13 대림동1차한신아파트                0
47                서울특별시 영등포구 신풍로 28 비스타동원                0
63                    서울특별시 영등포구 선유서로 116                0
143       서울특별시 영등포구 양산로25길 12 당산계룡리슈빌3단지                0
145       서울특별시 영등포구 양산로25길 16 당산계룡리슈빌1단지                0
146        서울특별시 영등포구 양산로25길 8 당산계룡리슈빌2단지                0
159                 서울특별시 영등포구 영등포로53길 40                0
220           서울특별시 영등포구 문래로 94-7 문래건영아파트                0
222        서울특별시 영등포구 가마산로 483 신길 AK 푸르지오                0
255                 서울특별시 영등포구 선유서로21길 30                0
270        

In [433]:
Dudemdvh # df_energy : cleaned_address / 전기사용량
df1 # display_address / 


   # df_energy: 전력소비량 정보 (address, electricity_usage)
Dudemdvh.rename(columns = {"cleaned_address" : "address",
                           "전기사용량" : "electricity_use"}, inplace=True)
   
   # df_building: 건물 정보 (address, building_info 등)
df1.rename(columns = {"display_address" : "address",
                                    "floor_count": "building_info"}, inplace=True)

In [434]:
geo_data = get_geo_data_naver(CLIENT_ID, CLIENT_SECRET, '영등포')

In [435]:
merged_df = geo_data.merge_data(Dudemdvh, df1, similarity_threshold=0.5)

In [439]:
merged_df["electricity_use"]

0            0.0
1            0.0
2        41503.0
3        41503.0
4            0.0
          ...   
10979        0.0
10980        0.0
10981        0.0
10982        0.0
10983        0.0
Name: electricity_use, Length: 10984, dtype: float64

In [440]:
def generate_naver_map_html(df, client_id, output_filename="search_map.html"):
    """
    네이버 지도에 검색창 + 자동완성 기능을 구현해 HTML 파일을 생성하는 함수.
    - df: 최소한 'lat', 'lon', 'address', 'electricity_usage' 컬럼을 가정
      ( address: 검색용 + 표시용, electricity_usage: 마커 클릭 시 InfoWindow에 표시 )
    - client_id: 네이버 지도 API 인증용 (ncpClientId)
    - output_filename: 생성할 HTML 파일명
    """

    # 1) 지도 중심 좌표 설정 (비어 있으면 기본값)
    center_lat = df["lat"].mean() if not df.empty else 37.507975
    center_lon = df["lon"].mean() if not df.empty else 126.910324

    # 2) df → Python dict 목록
    #    (각 행에 'address', 'lat', 'lon', 'electricity_usage'가 포함되어 있어야 함)
    markers = df.to_dict(orient="records")

    # 3) HTML 템플릿
    html_template = f"""
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <title>네이버 지도 - 주소 검색 & InfoWindow(전력소비량)</title>
        <script type="text/javascript" src="https://oapi.map.naver.com/openapi/v3/maps.js?ncpClientId={client_id}"></script>
        <style>
            body {{
                margin: 0; 
                padding: 0;
            }}
            #map {{
                width: 100%; 
                height: 100vh; 
            }}
            #searchBox {{
                position: absolute;
                top: 10px;
                left: 10px;
                width: 300px;
                padding: 10px;
                background: white;
                border: 1px solid #ccc;
                border-radius: 10px;
                z-index: 1000;
            }}
            #searchInput {{
                width: 100%;
                padding: 8px;
                font-size: 16px;
                border: 1px solid #ccc;
                border-radius: 5px;
            }}
            #suggestions {{
                list-style: none;
                padding: 0;
                margin: 5px 0 0 0;
                border: 1px solid #ccc;
                border-radius: 5px;
                overflow: hidden;
            }}
            #suggestions li {{
                padding: 8px;
                cursor: pointer;
                border-bottom: 1px solid #eee;
            }}
            #suggestions li:hover {{
                background: #e0f7fa;
            }}
        </style>
    </head>
    <body>
        <!-- 지도 영역 -->
        <div id="map"></div>

        <!-- 검색창 -->
        <div id="searchBox">
            <input type="text" id="searchInput" placeholder="주소를 검색하세요...">
            <ul id="suggestions"></ul>
        </div>

        <script>
            // 지도 초기화
            var mapOptions = {{
                center: new naver.maps.LatLng({center_lat}, {center_lon}),
                zoom: 12
            }};
            var map = new naver.maps.Map('map', mapOptions);

            // 파이썬에서 넘어온 markers 데이터
            var locations = {json.dumps(markers, ensure_ascii=False)};
            var currentMarker = null;

            // 검색 DOM 참조
            var searchInput = document.getElementById('searchInput');
            var suggestions = document.getElementById('suggestions');

            // 입력 이벤트 (부분 문자열 검색) -> address 필드 사용
            searchInput.addEventListener('input', function() {{
                var query = this.value.toLowerCase().trim();
                suggestions.innerHTML = '';

                if (query) {{
                    // address 필드에 부분 문자열이 포함된 항목만 필터
                    var filtered = locations.filter(function(item) {{
                        return item.address && item.address.toLowerCase().includes(query);
                    }});

                    // 최대 5개까지만 표시
                    filtered.slice(0, 5).forEach(function(item) {{
                        var li = document.createElement('li');
                        // 목록에는 주소가 뜨도록
                        li.textContent = item.address;
                        li.onclick = function() {{
                            // 항목 선택 시, 검색창에 주소를 넣음
                            searchInput.value = item.address;
                            suggestions.innerHTML = '';
                            showMarker(item);
                        }};
                        suggestions.appendChild(li);
                    }});
                }}
            }});

            // 선택된 항목의 lat, lon에 마커 표시
            function showMarker(item) {{
                // 기존 마커 제거
                if (currentMarker) {{
                    currentMarker.setMap(null);
                }}

                // 새 마커 생성
                currentMarker = new naver.maps.Marker({{
                    position: new naver.maps.LatLng(item.lat, item.lon),
                    map: map
                }});

                // InfoWindow: 주소 + 전력소비량
                var infoWindow = new naver.maps.InfoWindow({{
                    content: `
                        <div style="padding:8px; font-size:14px;">
                            <b>주소:</b> ${'{'}item.address{'}'}<br>
                            <b>전력소비량:</b> ${'{'}item.electricity_use{'}'} kWh
                        </div>
                    `
                }});

                // 마커 클릭 시 InfoWindow 열기
                naver.maps.Event.addListener(currentMarker, 'click', function() {{
                    infoWindow.open(map, currentMarker);
                }});

                // 지도 이동 + 확대
                map.setCenter(new naver.maps.LatLng(item.lat, item.lon));
                map.setZoom(17);
            }}
        </script>
    </body>
    </html>
    """

    # 4) HTML 파일로 저장
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(html_template)

    print(f"✅ 네이버 지도 HTML 파일이 생성되었습니다: {output_filename}")

def main():
    # 예시 데이터 (lat, lon, address, electricity_usage)
    # 실사용 시, merged_df 등 실제 데이터프레임 사용
    df = merged_df

    # 네이버 지도 클라이언트 ID
    client_id = CLIENT_ID  # 발급받은 ID로 대체

    # HTML 생성
    generate_naver_map_html(df, client_id, output_filename="search_map.html")

if __name__ == "__main__":
    main()


✅ 네이버 지도 HTML 파일이 생성되었습니다: search_map.html


In [441]:
merged_df[merged_df["electricity_use"]>0]

,Unnamed: 0,Unnamed: 0_x,ADRES,COMPLEX_GB_CD,COMPLEX_NM1,COMPLEX_NM2,COMPLEX_NM3,COMPLEX_PK,DONG_CNT,PNU,...,DONG_NM2,DONG_NM3,building_info,concrete_address,도로명주소,address,lat,lon,electricity_usage,electricity_use
2,335962,139516,서울특별시 영등포구 대림동 851-73,3,오름채,오름채,오름채,11560320342332,1,1156013300108510073,...,NaN,NaN,5,NaN,서울특별시 영등포구 대림로22길 7-1 오름채,서울특별시 영등포구 대림로22길 7-1 오름채,37.493798,126.901772,0,41503.0
3,335963,139516,서울특별시 영등포구 대림동 851-73,3,오름채,오름채,오름채,11560320342332,1,1156013300108510073,...,NaN,NaN,5,NaN,서울특별시 영등포구 대림로22길 7-1 오름채,서울특별시 영등포구 대림로22길 7-1 오름채,37.493798,126.901772,0,41503.0
8,335968,139519,서울특별시 영등포구 신길동 278-108,3,펠리체리움,펠리체리움,펠리체리움,11560320343155,1,1156013200102780108,...,NaN,NaN,4,NaN,서울특별시 영등포구 신길로41길 26 펠리체리움,서울특별시 영등포구 신길로41길 26 펠리체리움,37.508685,126.908349,0,93198.0
9,335969,139519,서울특별시 영등포구 신길동 278-108,3,펠리체리움,펠리체리움,펠리체리움,11560320343155,1,1156013200102780108,...,NaN,NaN,4,NaN,서울특별시 영등포구 신길로41길 26 펠리체리움,서울특별시 영등포구 신길로41길 26 펠리체리움,37.508685,126.908349,0,93198.0
12,335972,139521,서울특별시 영등포구 대림동 851-100,3,하나세인스톤,하나세인스톤,하나세인스톤 A동,11560320343157,2,1156013300108510100,...,A동,NaN,5,하나세인스톤 A동,서울특별시 영등포구 대림로22길 10 하나세인스톤,서울특별시 영등포구 대림로22길 10 하나세인스톤 A동,37.493890,126.902204,0,41503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10945,405079,167387,서울특별시 영등포구 당산동 121-228,3,비앤비퀸즈빌102동,비앤비퀸즈빌 102동,NaN,11560320339717,1,1156011700101210228,...,NaN,NaN,9,NaN,서울특별시 영등포구 국회대로55길 22 비앤비퀸즈빌 102동,서울특별시 영등포구 국회대로55길 22 비앤비퀸즈빌 102동,37.527212,126.907903,0,58258.0
10952,405086,167391,서울특별시 영등포구 대림동 776-9,3,광진아트빌5차,NaN,NaN,11560320339851,1,1156013300107760009,...,NaN,NaN,5,NaN,서울특별시 영등포구 가마산로32길 11,서울특별시 영등포구 가마산로32길 11,37.498257,126.895683,0,64400.0
10953,405087,167391,서울특별시 영등포구 대림동 776-9,3,광진아트빌5차,NaN,NaN,11560320339851,1,1156013300107760009,...,NaN,NaN,5,NaN,서울특별시 영등포구 가마산로32길 11,서울특별시 영등포구 가마산로32길 11,37.498257,126.895683,0,64400.0
10954,405088,167392,서울특별시 영등포구 당산동 121-109,3,하임4S,하임4 S,하임4 S,11560320340374,1,1156011700101210109,...,NaN,NaN,10,NaN,서울특별시 영등포구 국회대로55길 26 하임4 S,서울특별시 영등포구 국회대로55길 26 하임4 S,37.527498,126.907764,0,58258.0
